Generate RAW Data

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/shell-contest/Distance_Matrix.csv
/kaggle/input/shell-contest/sample_submission.csv
/kaggle/input/shell-contest/Biomass_History.csv


In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import requests
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error
import time
from datetime import datetime

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
biomass = pd.read_csv('/kaggle/input/shell-contest/Biomass_History.csv')
distance = pd.read_csv('/kaggle/input/shell-contest/Distance_Matrix.csv')
submission = pd.read_csv('/kaggle/input/shell-contest/sample_submission.csv')

In [4]:
biomass

,Index,Latitude,Longitude,2010,2011,2012,2013,2014,2015,2016,2017
0,0,24.66818,71.33144,8.475744,8.868568,9.202181,6.023070,10.788374,6.647325,7.387925,5.180296
1,1,24.66818,71.41106,24.029778,28.551348,25.866415,21.634459,34.419411,27.361908,40.431847,42.126945
2,2,24.66818,71.49069,44.831635,66.111168,56.982258,53.003735,70.917908,42.517117,59.181629,73.203232
3,3,24.66818,71.57031,59.974419,80.821304,78.956543,63.160561,93.513924,70.203171,74.536720,101.067352
4,4,24.66818,71.64994,14.653370,19.327524,21.928144,17.899586,19.534035,19.165791,16.531315,26.086885
...,...,...,...,...,...,...,...,...,...,...,...
2413,2413,20.15456,72.84432,5.199882,4.516778,4.321080,2.658953,5.113997,5.301668,6.419223,5.321604
2414,2414,20.15456,72.92394,0.122287,0.126717,0.101494,0.111509,0.121749,0.122467,0.145785,0.120626
2415,2415,20.15456,73.00357,0.039415,0.040843,0.032713,0.035941,0.039241,0.039473,0.046989,0.038879
2416,2416,20.15456,73.08319,2.719220,1.370163,0.818687,1.205721,1.316443,1.324201,1.576338,1.304297


In [5]:
submission

,year,data_type,source_index,destination_index,value
0,20182019,depot_location,1256,NaN,NaN
1,20182019,depot_location,1595,NaN,NaN
2,20182019,depot_location,1271,NaN,NaN
3,20182019,depot_location,2001,NaN,NaN
4,20182019,depot_location,2201,NaN,NaN
...,...,...,...,...,...
26654,2019,pellet_demand_supply,1791,1461.0,9441.860623
26655,2019,pellet_demand_supply,494,1465.0,3283.042945
26656,2019,pellet_demand_supply,494,1692.0,5161.343838
26657,2019,pellet_demand_supply,494,1535.0,8701.710618


In [6]:
columns = ['Index', 'Latitude', 'Longitude', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017']

In [7]:
def pivot_yearly(df,year):
    df = pd.DataFrame()
    df['Date']=year
    df[['Longitude','Latitude']] = biomass[['Longitude','Latitude']]
    df["Production"] = biomass[str(year)]
    df['Date']=int(year)
    return df

df_combined = pd.DataFrame()
for year in range(2010,2018):
    df_temp = pivot_yearly(biomass,year)
    df_combined = pd.concat([df_combined,df_temp],axis=0)

display(df_combined)

,Date,Longitude,Latitude,Production
0,2010,71.33144,24.66818,8.475744
1,2010,71.41106,24.66818,24.029778
2,2010,71.49069,24.66818,44.831635
3,2010,71.57031,24.66818,59.974419
4,2010,71.64994,24.66818,14.653370
...,...,...,...,...
2413,2017,72.84432,20.15456,5.321604
2414,2017,72.92394,20.15456,0.120626
2415,2017,73.00357,20.15456,0.038879
2416,2017,73.08319,20.15456,1.304297


In [8]:
df_combined.describe()

,Date,Longitude,Latitude,Production
count,19344.000000,19344.000000,19344.000000,19344.000000
mean,2013.500000,71.826215,22.506537,141.215293
std,2.291347,1.389552,1.034748,142.042419
min,2010.000000,68.624190,20.154560,0.000072
25%,2011.750000,70.774060,21.685970,24.860065
50%,2013.500000,71.888820,22.491970,109.626263
75%,2015.250000,73.003570,23.297970,207.746037
max,2017.000000,74.436820,24.668180,1232.853760


# External Data Integration

https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.41&start_date=2023-07-07&end_date=2023-07-21&hourly=temperature_2m,pressure_msl,windspeed_10m,windspeed_100m,winddirection_10m,winddirection_100m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm&daily=weathercode,temperature_2m_max,apparent_temperature_max,rain_sum,windspeed_10m_max&timezone=GMT

In [9]:
weather_daily_params = "temperature_2m_max,apparent_temperature_max,precipitation_sum,precipitation_hours,windspeed_10m_max"
weather_hourly_params = "temperature_2m,pressure_msl,windspeed_10m,windspeed_100m,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm"

In [10]:
# Function to fetch weather data for a given latitude and longitude
def fetch_weather_data(latitude, longitude, start_date, end_date):
    # Define the API endpoint URL
    api_url = "https://archive-api.open-meteo.com/v1/archive"

    # Define the API parameters
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "daily":weather_daily_params,
        "hourly":weather_hourly_params,
        "timezone": "GMT"
    }

    max_retries = 3  # Maximum number of retry attempts
    retry_delay = 2  # Time delay between retry attempts in seconds

    for retry in range(max_retries):
        # Send the API request and get the response
        response = requests.get(api_url, params=params)
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the response JSON
            weather_data = response.json()

            # Convert the JSON data to a DataFrame
            weather_df = pd.DataFrame(weather_data)
            #weather_df['time'] = pd.to_datetime(weather_df['time'])
            return weather_df

        elif response.status_code == 429:
            # If received 429 error, retry after a delay
            print(f"Rate limit exceeded. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)

        else:
            print(f"API request failed. Status Code:", response.status_code)
            print(latitude, longitude, start_date, end_date)
            return None

    print("Maximum number of retries reached. Unable to fetch weather data.")
    return None

In [11]:
unique_coordinates = biomass[['Latitude','Longitude']]
unique_coordinates

,Latitude,Longitude
0,24.66818,71.33144
1,24.66818,71.41106
2,24.66818,71.49069
3,24.66818,71.57031
4,24.66818,71.64994
...,...,...
2413,20.15456,72.84432
2414,20.15456,72.92394
2415,20.15456,73.00357
2416,20.15456,73.08319


In [12]:
import pandas as pd
from tqdm import tqdm

# Define your fetch_weather_data function here

def process_weather_data(weather_data):
    daily_weather = pd.DataFrame()
    hourly_weather = pd.DataFrame()
    rows = weather_data.index
    for c in rows:
        daily_weather[c] = weather_data['daily'][c]
        hourly_weather[c] = weather_data['hourly'][c]
    daily_weather['time'] = pd.to_datetime(daily_weather['time'])
    hourly_weather['time'] = pd.to_datetime(hourly_weather['time'])
    return daily_weather, hourly_weather

def fetch_and_save_weather_data(latitude, longitude, start_date, end_date, save_path):
    weather_data = fetch_weather_data(latitude, longitude, start_date, end_date)
    if weather_data is not None:
        daily_weather, hourly_weather = process_weather_data(weather_data)
        
        # Remove columns with mostly NaN values
        daily_weather = daily_weather.dropna(axis=1, thresh=len(daily_weather) // 2)
        hourly_weather = hourly_weather.dropna(axis=1, thresh=len(hourly_weather) // 2)
        
        daily_weather['Latitude'] = latitude
        daily_weather['Longitude'] = longitude
        hourly_weather['Latitude'] = latitude
        hourly_weather['Longitude'] = longitude
        return daily_weather, hourly_weather
    else:
        return None, None

In [13]:
# Specify the directory to save the data
save_path = 'weather_data'

# Initialize lists to store daily and hourly weather data
all_combined_data = []

for ind in tqdm(unique_coordinates.index):
    latitude = unique_coordinates['Latitude'][ind]
    longitude = unique_coordinates['Longitude'][ind]
    start_date = '2009-01-01'
    end_date = '2019-12-31'
    daily_data, hourly_data = fetch_and_save_weather_data(latitude, longitude, start_date, end_date, save_path)
    
    if daily_data is not None and hourly_data is not None:
        combined_data = pd.merge(daily_data, hourly_data, left_on='time', right_on='time')
        all_combined_data.append(combined_data)
    else:
        print(f"Failed to fetch data for Latitude: {latitude}, Longitude: {longitude}")

100%|██████████| 2418/2418 [3:39:55<00:00,  5.46s/it]


In [14]:
# Concatenate all combined data
combined_weather_data = pd.concat(all_combined_data, ignore_index=True)

# Save the combined data to a CSV file
combined_weather_data.to_csv('combined_weather_data.csv', index=False)

combined_weather_data

,time,temperature_2m_max,apparent_temperature_max,precipitation_sum,precipitation_hours,windspeed_10m_max,Latitude_x,Longitude_x,temperature_2m,pressure_msl,windspeed_10m,windspeed_100m,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,Latitude_y,Longitude_y
0,2009-01-01,28.0,24.6,0.0,0.0,14.3,24.66818,71.33144,13.9,1015.5,14.3,26.6,24.8,28.9,0.109,24.66818,71.33144
1,2009-01-02,27.9,25.7,0.0,0.0,10.1,24.66818,71.33144,13.4,1014.5,9.7,16.9,24.8,28.8,0.109,24.66818,71.33144
2,2009-01-03,28.1,25.5,0.0,0.0,12.8,24.66818,71.33144,12.1,1014.3,9.2,8.8,24.7,28.7,0.109,24.66818,71.33144
3,2009-01-04,27.1,26.1,0.0,0.0,18.4,24.66818,71.33144,16.5,1013.6,12.3,23.8,24.6,28.7,0.109,24.66818,71.33144
4,2009-01-05,24.6,23.3,0.0,0.0,20.0,24.66818,71.33144,17.2,1012.7,18.9,32.1,24.6,28.6,0.109,24.66818,71.33144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9713101,2019-12-27,30.6,31.6,0.0,0.0,12.8,20.15456,73.16282,19.0,1012.4,8.3,18.9,26.8,27.2,0.263,20.15456,73.16282
9713102,2019-12-28,30.9,31.8,0.0,0.0,13.7,20.15456,73.16282,19.7,1011.6,12.3,25.2,26.7,27.2,0.262,20.15456,73.16282
9713103,2019-12-29,29.6,30.7,0.0,0.0,12.2,20.15456,73.16282,21.3,1012.2,11.4,23.5,26.6,27.2,0.260,20.15456,73.16282
9713104,2019-12-30,28.8,30.7,0.0,0.0,12.5,20.15456,73.16282,20.1,1011.7,6.0,14.2,26.6,27.2,0.259,20.15456,73.16282
